In [1]:
import pandas as pd
import numpy as np
gunao = pd.read_csv('gunao_surface.csv')
tikob = pd.read_csv('tikub_surface_bottom.csv')

In [2]:
tikob_surface = tikob[tikob['COLLECTION'] == 'Surface']

In [3]:
columns_to_exclude = ['DATE', 'MONTH', 'YEAR', 'STATION', 'REPLICATE', 'COLLECTION', 'Latitude', 'Longtitude']
tikob_fil = tikob_surface.drop(columns=columns_to_exclude)
gunao_fil = gunao.drop(columns=columns_to_exclude)

In [4]:
# Check for missing values in gunao
missing_values_gunao = gunao_fil.isnull().sum()
print("Missing values in gunao_surface.csv:")
print(missing_values_gunao[missing_values_gunao > 0])

# Check for missing values in tikub
missing_values_tikub = tikob_fil.isnull().sum()
print("\nMissing values in tikub_surface_bottom.csv:")
print(missing_values_tikub[missing_values_tikub > 0])

Missing values in gunao_surface.csv:
Series([], dtype: int64)

Missing values in tikub_surface_bottom.csv:
Series([], dtype: int64)


In [5]:
from sklearn.model_selection import train_test_split
feature_columns = [
    'pH','DO (mg/L)','TDS (mg/L)','Salinity (ppt)','Cond (uS/cm)','Temp (°C)','TSS (mg/L)', 'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)',
    'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 'Chlorophyll (ug/L)',
    'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)',
    'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
]
target_column = 'BOD (mg/L)'

X_tikob = tikob_fil[feature_columns]
y_tikob = tikob_fil[target_column]

X_train_tikob, X_test_tikob, y_train_tikob, y_test_tikob = train_test_split(X_tikob, y_tikob, test_size=0.2, random_state=42)

train_data = pd.concat([X_train_tikob, y_train_tikob], axis=1)
test_data = pd.concat([X_test_tikob, y_test_tikob], axis=1)

train_data.to_csv('tikub_surface_train.csv', index=False)
test_data.to_csv('tikub_surface_test.csv', index=False)

In [6]:
X_gunao = gunao_fil[feature_columns]
y_gunao = gunao_fil[target_column]

#X_train, X_test, y_train, y_test = train_test_split(X_gunao, y_gunao, test_size=0.2, random_state=42)
X_train_gunao, X_test_gunao, y_train_gunao, y_test_gunao = train_test_split(X_gunao, y_gunao, test_size=0.2, random_state=42)

train_data = pd.concat([X_train_gunao, y_train_gunao], axis=1)
test_data = pd.concat([X_test_gunao, y_gunao], axis=1)

train_data.to_csv('gunao_fil_train.csv', index=False)
test_data.to_csv('gunao_fil_test.csv', index=False)

In [10]:
pip install scikit-optimize


     -------------------------------------- 107.8/107.8 kB 1.5 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


**SVR**

In [22]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV
import joblib

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Handle missing values if any
X_combined.fillna(X_combined.mean(), inplace=True)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_combined, test_size=0.2, random_state=2)

# Define the parameter search space for Bayesian optimization
param_space = {
    'C': (1e-6, 1e+6, 'log-uniform'),
    'epsilon': (1e-6, 1e-1, 'log-uniform'),
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': (1e-6, 1e+1, 'log-uniform')
}

# Initialize SVR model
svr = SVR()

# Perform Bayesian optimization with cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=2)
bayes_search = BayesSearchCV(svr, param_space, n_iter=100, cv=kf, scoring='neg_mean_squared_error', random_state=2)
bayes_search.fit(X_train, y_train)

# Get the best model
best_svr = bayes_search.best_estimator_

# Make predictions on the test set
predictions = best_svr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print('Best hyperparameters:', bayes_search.best_params_)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAE:', mae)
print('Test R^2:', r2)
print('Test MAPE:', mape)
joblib.dump(best_svr, 'best_SRV_bod_model.pkl')


Best hyperparameters: OrderedDict([('C', 1000000.0), ('epsilon', 1e-06), ('gamma', 0.07351590534406176), ('kernel', 'rbf')])
Test MSE: 0.4553891539849098
Test RMSE: 0.6748252766345595
Test MAE: 0.4745498614234804
Test R^2: 0.7345415703431775
Test MAPE: 45.63750080860805


['best_SRV_bod_model.pkl']

**RANDOM FOREST**

In [20]:
joblib.dump(best_svr, 'best_SRV_bod_model.pkl')


['best_SRV_bod_model.pkl']

In [21]:
import pandas as pd
import joblib

# Function to load the model and predict BOD from a CSV file
def predict_bod_from_csv(input_csv):
    # Load the trained model
    model = joblib.load('best_SRV_bod_model.pkl')
    
    # Define the feature columns
    feature_columns = [
        'pH', 'DO (mg/L)', 'TDS (mg/L)', 'Salinity (ppt)', 'Cond (uS/cm)', 'Temp (°C)', 'TSS (mg/L)', 
        'NO2 (ppm)', 'NO3 (ppm)', 'PO4  (ppm)', 'NH4 (ppm)', 'TN (ppm)', 'TP (ppm)', 'BGA-PC (ug/L)', 
        'Chlorophyll (ug/L)', 'Turbidity (FNU)', 'Coliform (CFU/100ml)', 'Cu (ppm)', 'Fe (ppm)', 
        'Mn(ppm)', 'Zn(ppm)', 'Cr(ppm)', 'Cd(ppm)', 'Hg(ppm)', 'As(ppm)', 'Pb(ppm)'
    ]

    # Read the input data from CSV
    input_df = pd.read_csv(input_csv)

    # Ensure the input data has all necessary columns
    missing_cols = set(feature_columns) - set(input_df.columns)
    if missing_cols:
        raise ValueError(f"Missing columns in input data: {missing_cols}")

    # Predict BOD
    predictions = model.predict(input_df[feature_columns])
    
    # Print only the predictions
    for prediction in predictions:
        print(prediction)

# Example usage
input_csv = 'Book1.csv'  # Replace with the path to your CSV file
predict_bod_from_csv(input_csv)


2.3254945948300914


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV
import joblib

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=None)

# Define the parameter search space for Bayesian optimization
param_space = {
    'n_estimators': (113, 500),               # Number of trees
    'max_features': (0.1, 0.9, 'uniform'),    # Fraction of features to consider at each split
    'max_depth': (30, 50),                    # Maximum depth of the tree
    'min_samples_split': (2, 10),             # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 10)               # Minimum number of samples required to be at a leaf node
}

# Initialize Random Forest Regressor model
rf = RandomForestRegressor(random_state=10)

# Perform Bayesian optimization with cross-validation
bayes_search = BayesSearchCV(rf, param_space, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=None)
bayes_search.fit(X_train, y_train)

# Get the best model
best_rf = bayes_search.best_estimator_

# Make predictions on the test set
predictions = best_rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print('Best hyperparameters:', bayes_search.best_params_)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAE:', mae)
print('Test R^2:', r2)
print('Test MAPE:', mape)

# Save the best model
joblib.dump(best_rf, 'best_rf_bod_model.pkl')



KeyboardInterrupt



**RIDGE / Multilinear**

In [15]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV
import joblib
# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=None)

# Define the parameter search space for Bayesian optimization
param_space = {
    'alpha': (1e-54, 1e-20, 'log-uniform'),
    'tol' : (1e-10, 1e-5)
}

# Initialize Ridge regression model
ridge = Ridge(random_state=None, solver='svd')

# Perform Bayesian optimization with cross-validation
bayes_search = BayesSearchCV(ridge, param_space, n_iter=100, cv=10, scoring='neg_mean_squared_error', random_state=None)
bayes_search.fit(X_train, y_train)

# Get the best model
best_ridge = bayes_search.best_estimator_

# Make predictions on the test set
predictions = best_ridge.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print('Best hyperparameters:', bayes_search.best_params_)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAE:', mae)
print('Test R^2:', r2)
print('Test MAPE:', mape)

joblib.dump(best_ridge, 'best_ridge_bod_model(2).pkl')


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1.8676340651560328e-40, 2.011264239573854e-06] before, using random point [1.0071301280789945e-31, 1.975455885059174e-06]
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1.827002759868189e-49, 9.994199795038181e-06] before, using random point [3.764238172042555e-38, 7.6041870687078714e-06]
  warnings.warn(


Best hyperparameters: OrderedDict([('alpha', 1e-20), ('tol', 1e-05)])
Test MSE: 1.0730946718072671
Test RMSE: 1.035902829326799
Test MAE: 0.7822572876510505
Test R^2: 0.36736909506648674
Test MAPE: 59.241389905136465


['best_ridge_bod_model(2).pkl']

In [46]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
import joblib
# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=30)

# Define a pipeline with scaling, polynomial features, and ridge regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('ridge', Ridge(random_state=30))
])

# Define the parameter space including polynomial degree and interaction only
param_space = {
    'poly__degree': [1, 2],
    'ridge__alpha': (1e-50, 1e-5, 'log-uniform'),
    'ridge__tol': (1e-60, 1e-30, 'log-uniform'),
    'ridge__solver': ['svd']
}

# Perform Bayesian optimization with cross-validation
bayes_search = BayesSearchCV(pipeline, param_space, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=30)
bayes_search.fit(X_train, y_train)

# Get the best model
best_pipeline = bayes_search.best_estimator_

# Make predictions on the test set
predictions = best_pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print('Best hyperparameters:', bayes_search.best_params_)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAE:', mae)
print('Test R^2:', r2)
print('Test MAPE:', mape)

joblib.dump(best_pipeline, 'best_ridge_bod_model_with_pipeline.pkl')


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\skopt\space\space.py:116: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to 

Best hyperparameters: OrderedDict([('poly__degree', 1), ('ridge__alpha', 2.854967964998318e-41), ('ridge__solver', 'svd'), ('ridge__tol', 1e-60)])
Test MSE: 0.7396676268506662
Test RMSE: 0.8600393170376958
Test MAE: 0.6687305706262144
Test R^2: 0.558936107897051
Test MAPE: 59.59661186300743


['best_ridge_bod_model_with_pipeline.pkl']

In [48]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV
import joblib

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Define the parameter search space for Bayesian optimization
param_space = {
    'alpha': (1e-54, 1e-20, 'log-uniform'),
    'tol' : (1e-10, 1e-5)
}

# Initialize Ridge regression model
ridge = Ridge(random_state=None)

# Function to evaluate model performance
def evaluate_model(random_seed):
    # Split the combined dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=random_seed)

    # Perform Bayesian optimization with cross-validation
    bayes_search = BayesSearchCV(ridge, param_space, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=random_seed)
    bayes_search.fit(X_train, y_train)

    # Get the best model
    best_ridge = bayes_search.best_estimator_

    # Make predictions on the test set
    predictions = best_ridge.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

    return mse, rmse, mae, r2, mape, bayes_search.best_params_, best_ridge

# Iterate over a range of random seeds and find the best one
best_seed = None
best_score = float('inf')
best_model = None
best_params = None
for seed in range(101, 501):
    mse, rmse, mae, r2, mape, best_params_seed, best_ridge_seed = evaluate_model(seed)
    if mse < best_score:
        best_score = mse
        best_seed = seed
        best_model = best_ridge_seed
        best_params = best_params_seed
    print(f'Seed: {seed}, MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R^2: {r2}, MAPE: {mape}')

print('Best Seed:', best_seed)
print('Best hyperparameters:', best_params)
print('Best Test MSE:', best_score)

# Save the best model
joblib.dump(best_model, 'best_ridge_bod_model.pkl')


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.93774e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.17064e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.93774e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.17064e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:21

Seed: 101, MSE: 1.193712139577878, RMSE: 1.092571343015127, MAE: 0.8966613124198733, R^2: 0.47951435141273446, MAPE: 77.33556241462341


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.38901e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.38901e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.38901e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.38901e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:21

Seed: 102, MSE: 0.9769629412158747, RMSE: 0.9884143570466157, MAE: 0.7978667618678167, R^2: 0.4809524468974802, MAPE: 118.7389406775606


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.53342e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.53342e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.53342e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.53342e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:21

...

Seed: 181, MSE: 250.8698551778374, RMSE: 15.838871651031123, MAE: 2.6181521926034246, R^2: -149.88380806915694, MAPE: 182.61021608308602


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.50373e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.13269e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.50373e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.13269e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:21

Seed: 182, MSE: 1.0293896863895309, RMSE: 1.014588432020359, MAE: 0.7801877498554252, R^2: 0.5665865373520289, MAPE: 54.57604405411261


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.44632e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.12215e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.44632e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=4.12215e-23): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\linear_model\_ridge.py:21

KeyboardInterrupt: 

**MULTI-LAYERED PERCEPTRON**

In [55]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from skopt import BayesSearchCV

# Combine the datasets
X_combined = pd.concat([X_tikob, X_gunao], axis=0)
y_combined = pd.concat([y_tikob, y_gunao], axis=0)

# Split the combined dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Define the parameter search space for Bayesian optimization
param_space = {
    'hidden_layer_sizes': (50, 200),  # Range of sizes for hidden layers
    'activation': ['relu', 'tanh'],   # Activation functions
    'alpha': (1e-6, 1e-2, 'log-uniform'),  # L2 regularization parameter
    'learning_rate': ['constant', 'invscaling', 'adaptive']  # Learning rate schedule
}

# Initialize MLP Regressor model
mlp = MLPRegressor(random_state=42)

# Perform Bayesian optimization with cross-validation
bayes_search = BayesSearchCV(mlp, param_space, n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42)
bayes_search.fit(X_train, y_train)

# Get the best model
best_mlp = bayes_search.best_estimator_

# Make predictions on the test set
predictions = best_mlp.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100

print('Best hyperparameters:', bayes_search.best_params_)
print('Test MSE:', mse)
print('Test RMSE:', rmse)
print('Test MAE:', mae)
print('Test R^2:', r2)
print('Test MAPE:', mape)


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multi

Best hyperparameters: OrderedDict([('activation', 'tanh'), ('alpha', 0.0016920625808447933), ('hidden_layer_sizes', 100), ('learning_rate', 'adaptive')])
Test MSE: 1.4313566451177167
Test RMSE: 1.1963931816579851
Test MAE: 0.9013865828801102
Test R^2: 0.4306289297277066
Test MAPE: 62.24097483045426


H:\BIOCHEMICAL\BOD_least\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
